## Visualization

In [1]:
!pip install geemap folium google-cloud-storage rasterio matplotlib > /dev/null 2>&1

In [2]:
import ee
import geemap
import os
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import rasterio
import folium
from folium.plugins import TimeSliderChoropleth
from branca.colormap import LinearColormap

drive.mount('/content/drive')
work_dir = '/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/'
os.makedirs(work_dir, exist_ok=True)


MessageError: Error: credential propagation was unsuccessful

In [ ]:
ee.Authenticate()
ee.Initialize(project='manifest-pride-258211')

In [ ]:
# Drive'dan TIFF dosyalarını yükle
lst_2015 = os.path.join(work_dir, 'LST_2015_Hamburg.tif')
lst_2024 = os.path.join(work_dir, 'LST_2025_Hamburg.tif')

In [ ]:
# Google Drive'dan TIFF'leri oku
def load_tiff(path):
    with rasterio.open(path) as src:
        return src.read(1), src.bounds, src.crs

lst_2015, bounds_2015, crs = load_tiff('/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/LST_2015_Hamburg.tif')
lst_2025, bounds_2025, _ = load_tiff('/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/LST_2025_Hamburg.tif')

In [ ]:
# LST verilerini normalize et (0-1 aralığında)
def normalize(data):
    return (data - np.nanmin(data)) / (np.nanmax(data) - np.nanmin(data))

lst_2015_norm = normalize(lst_2015)
lst_2025_norm = normalize(lst_2025)

# TimeSlider için GeoJSON formatına dönüştür
features = []
for idx, (year, data) in enumerate(zip([2015, 2025], [lst_2015_norm, lst_2025_norm])):
    for y in range(data.shape[0]):
        for x in range(data.shape[1]):
            if not np.isnan(data[y, x]):  # NaN değerleri atla
                features.append({
                    "type": "Feature",
                    "geometry": {
                        "type": "Point",
                        "coordinates": [
                            bounds_2015.left + (x / data.shape[1]) * (bounds_2015.right - bounds_2015.left),
                            bounds_2015.bottom + (y / data.shape[0]) * (bounds_2015.top - bounds_2015.bottom)
                        ]
                    },
                    "properties": {
                        "time": year,
                        "value": float(data[y, x]),
                        "style": {"color": ""}  # Renk sonradan eklenecek
                    }
                })

# GeoJSON oluştur
geojson = {"type": "FeatureCollection", "features": features}

In [ ]:
# Renk paleti (kırmızı-sıcak, mavi-soğuk)
colormap = LinearColormap(
    colors=['blue', 'green', 'yellow', 'red'],
    vmin=0, vmax=1,
    caption='LST (Normalize)'
)

# Harita başlat
m = folium.Map(
    location=[53.55, 9.99],
    zoom_start=12,
    tiles='CartoDB dark_matter'  # Koyu tema (heatmap için uygun)
)

# TimeSlider ekle
TimeSliderChoropleth(
    data=geojson,
    styledict={
        "fillColor": lambda feature: colormap(feature["properties"]["value"]),
        "weight": 0,
        "fillOpacity": 0.6
    },
    name="LST Time Series"
).add_to(m)

# Renk paletini haritaya ekle
colormap.add_to(m)

# Layer kontrolü
folium.LayerControl().add_to(m)
m

NameError: name 'LinearColormap' is not defined

In [ ]:
import geemap

# GEE'den doğrudan görüntüleri yükle (veya Drive'dan TIFF'leri geemap ile açın)
Map = geemap.Map(center=[53.55, 9.99], zoom=12)

# LST katmanlarını ekle
Map.add_raster('/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/LST_2015_Hamburg.tif', layer_name='LST 2015', colormap='inferno')
Map.add_raster('/content/drive/MyDrive/UHI-Detection-Analysis/data/raw//LST_2025_Hamburg.tif', layer_name='LST 2025', colormap='inferno')

# TimeSlider için katmanları birleştir
Map.add_time_slider(
    [lst_2015, lst_2025],  # Raster listesi
    labels=['2015', '2025'],  # Zaman etiketleri
    colormap='inferno',
    time_interval=2  # Slider hızı (saniye)
)
Map

ImportError: localtileserver is not installed. Please install it before proceeding. https://github.com/banesullivan/localtileserver

In [ ]:
!pip install nbstripout > /dev/null 2>&1

In [ ]:
nbstripout /content/drive/MyDrive/notebooks/07_visualization_deploy.ipynb
